In [37]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json

sys.path.append(str(Path("..").resolve()))
from src import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Migrate MySQL to MongoDB
As the database schema has evolved significantly since the current MySQL database was created, we've first migrated a subset of the collections using [_MongoDB Relational Migrator_](https://www.mongodb.com/resources/solutions/use-cases/mysql-to-mongodb).
The database snapshot up to this point can be loaded from `backup/1_after_migration` using the following cell.

In this notebook we are going to complete the migration using handwritten queries.

## Load post-migration data
The following cell reset the `omero_museum` database loading the `1_after_migration` snapshot.

In [38]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/1_after_migration/omero_museum"

## Visualizing the migrated database

In [39]:
schema=json.load(open("schema.json"))
connector=MongoDBConnector("omero_museum")
db=connector.db

The collections of the omero_museum db are:
----------------------------------------
[artisti]: _id cognome data_nasc id luogo_nasc nome sesso
[biglietti]: _id costo data_stampa evento gruppo id
[clienti]: _id cellulare cognome email id name  tariffa tessera
[dipendenti]: _id cellulare cognome curriculum data_nasc data_registrazione email id luogo_nasc nome sesso
[laboratori]: _id costo_classe costo_persona durata nome tipologia
[opere]: _id data descrizione id sala tipologia titolo
[questionari]: _id accompagnatori_visita data_compilazione motivazione_visita numero_visite ritorno tipologia_visita titolo_studi valutazione_esperienza valutazione_struttura valutazione_visita visitatore
[reparti]: _id nome piano posti_occ posti_tot stanza


There are 3 kinds of problems that catch the eye:
1. Collections need to be **renamed**,
2. Some collections are **missing**.
3. Several fields have **changed** during the design.

### 1. Renaming fields

In [40]:
collections_renaming = {
    "biglietti": "tickets",
    "reparti": "departments",
    "dipendenti": "roles",
    "opere": "artworks",
    "clienti": "visitors",
    "laboratori": "workshops",
    "questionari": "surveys",
    "artisti": "authors",
}
for old_name, new_name in collections_renaming.items():
    if old_name not in connector.collections:
        continue
    db[old_name].rename(new_name)
    cprint("Renaming collection", f"red:{old_name} --> {new_name}")

Renaming collection biglietti --> tickets
Renaming collection reparti --> departments
Renaming collection dipendenti --> roles
Renaming collection opere --> artworks
Renaming collection clienti --> visitors
Renaming collection laboratori --> workshops
Renaming collection questionari --> surveys
Renaming collection artisti --> authors


### 2. Adding missing collections

In [41]:
collections_missing = [
    "rooms",
    "messages",
    "suppliers",
    "limited_events",
    "activities",
]
for name in collections_missing:
    db.create_collection(name)
    cprint("Creating collection", f"green:{name}")

Creating collection rooms
Creating collection messages
Creating collection suppliers
Creating collection limited_events
Creating collection activities


### 3. Renaming documents' entries

In [42]:
entries_renaming = {
    "artworks": {
        "data": "date",
        "descrizione": "description",
        "sala": "room",
        "tipologia": "type",
        "titolo": "title",
    },
    "authors": {
        "cognome": "surname",
        "data_nasc": "birth_date",
        "luogo_nasc": "home_town",
        "nome": "name",
        "sesso": "gender",
    },
    "departments": {
        "nome": "name",
        "piano": "floor",
        "posti_occ": "free_spots",
        "stanza": "room",
    },
    "roles": {
        "cellulare": "phone_number",
        "cognome": "surname",
        "curriculum": "curriculum",
        "data_nasc": "birth_date",
        "data_registrazione": "date_start",
        "email": "email",
        "luogo_nasc": "hometown",
        "nome": "name",
        "sesso": "gender",
    },
    "surveys": {
        "accompagnatori_visita": "accompanying_persons_visit",
        "data_compilazione": "date_of_compilation",
        "motivazione_visita": "reason_for_visit",
        "numero_visite": "number_of_visits",
        "ritorno": "return",
        "tipologia_visita": "type_of_visit",
        "titolo_studi": "title_of_studies",
        "valutazione_esperienza": "evaluation_of_experience",
        "valutazione_struttura": "evaluation_of_facility",
        "valutazione_visita": "evaluation_of_visit",
    },
    "tickets": {
        "costo": "price",
        "data_stampa": "date",
    },
    "visitors": {
        "cellulare": "cell phone number",
        "cognome": "surname",
        "email": "email",
        "name": "name",
        "tariffa": "fare",
    },
    "workshops": {
        "costo_classe": "price_class",
        "costo_persona": "price_person",
        "durata": "duration",
        "nome": "title",
        "tipologia": "type",
    },
}
for coll_name, mapping in tqdm(entries_renaming.items()):
    coll = db[coll_name]

    for doc in coll.find({}):
        new_doc = {"_id": doc["_id"]}
        for old_key, new_key in mapping.items():
            if old_key in doc:
                new_doc[new_key] = doc[old_key]

        coll.replace_one({"_id": doc["_id"]}, new_doc)  

100%|██████████| 8/8 [00:00<00:00, 43.93it/s]


In [43]:
connector.stats()

The collections of the omero_museum db are:
----------------------------------------
[activities]:
[artworks]: _id date description room title type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots name room
[limited_events]:
[messages]:
[roles]: _id birth_date curriculum date_start email gender hometown name phone_number surname
[rooms]:
[suppliers]:
[surveys]: _id accompanying_persons_visit date_of_compilation evaluation_of_experience evaluation_of_facility evaluation_of_visit number_of_visits reason_for_visit return title_of_studies type_of_visit
[tickets]: _id date price
[visitors]: _id cell phone number email fare surname
[workshops]: _id duration price_class price_person title type


## Setting right `_id` values
Some collections have a wrong primary key.

In [44]:
collections_id = {"workshops": "title", "artworks": "title", "departments": "name"}
for coll, key in collections_id.items():
    for doc in db[coll].find({}):
        id = doc.pop(key)
        db[coll].delete_one({"_id": doc["_id"]})
        db[coll].insert_one({**doc, "_id": id})
        cprint(f"[{coll}]:", f"red:{doc['_id']}", "-->", f"green:{id}")
    print("-" * 30)

[workshops]: 68b572f8845fcdbabc1c8941 --> Bestiario immaginario
[workshops]: 68b572f8845fcdbabc1c8942 --> Conosci Louis?
[workshops]: 68b572f8845fcdbabc1c8943 --> Contemporaneamente di-segno
[workshops]: 68b572f8845fcdbabc1c8944 --> Di-segno
[workshops]: 68b572f8845fcdbabc1c8945 --> Il Museo delle meraviglie
[workshops]: 68b572f8845fcdbabc1c8946 --> Impronte
[workshops]: 68b572f8845fcdbabc1c8947 --> Le cose raccontano storie
[workshops]: 68b572f8845fcdbabc1c8948 --> Libri tattili
[workshops]: 68b572f8845fcdbabc1c8949 --> Mini corso di ceramica
[workshops]: 68b572f8845fcdbabc1c894a --> Ri-tratto con tatto
[workshops]: 68b572f8845fcdbabc1c894b --> Ricordi da toccare
[workshops]: 68b572f8845fcdbabc1c894c --> Uno solo... ma tanti
------------------------------
[artworks]: 68b572f8845fcdbabc1c894d --> Il Partenone
[artworks]: 68b572f8845fcdbabc1c894e --> Il Discobolo
[artworks]: 68b572f8845fcdbabc1c894f --> Santa Maria del Fiore di Firenze
[artworks]: 68b572f8845fcdbabc1c8950 --> Duomo di M

---
## Dump Final Database

In [45]:
%%capture
!mongodump --host localhost:27017 --db omero_museum --out "../backup/2_migration_preprocessed"